In [8]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from semi_supervised_transformers import PreprocessingPipeline
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.preprocessing import LabelEncoder

In [9]:
data = pd.read_csv('/Users/annabzinkowska/DTU/master_thesis/data/processed_data_all_bertopic.csv')

/var/folders/n9/qhmmsj1s7w52648j4cjf7ztc0000gn/T/ipykernel_75839/639417878.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/annabzinkowska/DTU/master_thesis/data/processed_data_all_bertopic.csv')


In [10]:
data.head()

,product_id,description,month_mode,quarter_mode,year_mode,day_week_mode,quantity_sum,price_sum,unit_weight,customer_country_mode,customer_country_count,customer_id_count,category,unit_price_mean,description_original,topic,category_bertopic
0,010004,cfcf sfy bolted hanger nptf stud viton,2,1,2020,2,6,232.92,0.124,DK,1,1,-1,38.8200,"C24FCF-1/2-SFY-S BOLTED HANGER, N...",-1,-1
1,010005,cfcf sfy bolted hanger nptf stud viton,10,4,2019,2,200,1774.00,0.259,CO,1,1,-1,8.8700,"C24FCF-1-SFY-S BOLTED HANGER, N...",-1,-1
2,010006,cfcf sfy bolted hanger nptf stud viton,10,4,2019,2,300,1848.00,0.276,CO,1,1,-1,6.1600,"C24FCF-1 1/2-SFY-S BOLTED HANGER, N...",-1,-1
3,010015,cfcf sfy bolted hanger nptf stud viton,1,1,2019,2,300,2115.00,0.318,SE,2,2,-1,7.0500,"C24FCF-2-SFY-S BOLTED HANGER, N...",-1,-1
4,010040,cfcf sfy bolted hanger nptf stud viton,1,1,2019,2,400,3119.00,0.363,CO,2,2,-1,7.7975,"C24FCF-3-SFY-S BOLTED HANGER, N...",-1,-1


In [11]:
categorical_columns = ['month_mode', 'quarter_mode','year_mode','day_week_mode', 'customer_country_mode']
numerical_columns = ['quantity_sum', 'price_sum', 'unit_weight', 'unit_price_mean', 'customer_country_count', 'customer_id_count'] 

This code will train each classifier using the self-training approach, evaluate it on the validation set, and then display a ranking of classifiers by accuracy.

In [12]:
data['category_bertopic'].value_counts()

category_bertopic
-1    16969
C     11939
B      3513
A      2505
E       154
D       106
Name: count, dtype: int64

In [13]:
# Assuming `data` is your DataFrame and `categorical_columns`, `numerical_columns`, 
# and `log_transform_columns` are lists of column names

# Set a random seed for replicability
random_seed = 42

# Splitting data into labeled and unlabeled samples
labeled_data = data[data['category_bertopic'] != '-1']
unlabeled_data = data[data['category_bertopic'] == '-1']

X_labeled = labeled_data.drop(columns='category_bertopic')
y_labeled = labeled_data['category_bertopic']
X_unlabeled = unlabeled_data.drop(columns='category_bertopic')

# Further split labeled data for evaluation
X_train, X_val, y_train, y_val = train_test_split(X_labeled, y_labeled, test_size=0.2, stratify=y_labeled, random_state=random_seed)

# Check lengths
print("Length of X_train:", len(X_train))
print("Length of y_train:", len(y_train))
print("Length of X_val:", len(X_val))
print("Length of y_val:", len(y_val))

# Preprocess the data
pipeline = PreprocessingPipeline(categorical_columns, numerical_columns, text_column='description')
X_train_processed = pipeline.fit_transform(X_train, include_text=True)
X_val_processed = pipeline.transform(X_val, include_text=True)
X_unlabeled_processed = pipeline.transform(X_unlabeled, include_text=True)

# Double-check lengths after processing
print("Length of X_train_processed:", len(X_train_processed))
print("Length of X_val_processed:", len(X_val_processed))
print("Length of X_unlabeled_processed:", len(X_unlabeled_processed))

# Define the classifiers
classifiers = [
   ('Random Forest', RandomForestClassifier(random_state=random_seed)),
   ('XGBoost', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=random_seed)),
   ('LightGBM', lgb.LGBMClassifier(random_state=random_seed)),
   ('SVM', SVC(probability=True, random_state=random_seed)),
   ('KNN', KNeighborsClassifier()),
   ('Neural Network (MLP)', MLPClassifier(max_iter=1000, random_state=random_seed)),
]

# Fit the LabelEncoder on labels from the labeled dataset
le = LabelEncoder()
le.fit(y_labeled)

# Encode the labeled data
y_train_encoded = le.transform(y_train)
y_val_encoded = le.transform(y_val)

# Define a placeholder value for unlabeled data
# We'll use the next integer after the maximum label for labeled data
unlabeled_placeholder = len(le.classes_)

# Confidence threshold for self-training, default 0.75
threshold = 0.80

# Track the number of samples pseudo-labeled in each iteration
pseudo_labeled_counts = []

# Training and evaluation
results = []

Length of X_train: 14573
Length of y_train: 14573
Length of X_val: 3644
Length of y_val: 3644


/Users/annabzinkowska/anaconda3/envs/thesis_/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Length of X_train_processed: 14573
Length of X_val_processed: 3644
Length of X_unlabeled_processed: 16969


In [14]:
for name, base_classifier in classifiers:
    if hasattr(base_classifier, 'predict_proba'):
        self_training_model = SelfTrainingClassifier(base_classifier, criterion='threshold', threshold=threshold)

        # Combine the data
        combined_X = np.vstack((X_train_processed, X_unlabeled_processed))
        combined_y = np.concatenate([y_train_encoded, [-1] * len(X_unlabeled_processed)])

        # Track the number of samples pseudo-labeled in this iteration
        pseudo_labels = np.full(len(X_unlabeled_processed), -1)  # Placeholder labels for unlabeled data
        self_training_model.fit(combined_X, combined_y)

        # Get the model's confidence scores for the unlabeled data
        confidence_scores = self_training_model.predict_proba(X_unlabeled_processed)

        # Check which samples meet the confidence threshold for pseudo-labeling
        confident_samples = np.max(confidence_scores, axis=1) >= threshold

        # Update the pseudo-labels for confident samples
        pseudo_labels[confident_samples] = self_training_model.classes_[np.argmax(confidence_scores, axis=1)][confident_samples]

        # Count the number of pseudo-labeled samples in this iteration
        pseudo_labeled_count = np.sum(pseudo_labels != -1)
        pseudo_labeled_counts.append((name, pseudo_labeled_count))

        # Update the combined_y with pseudo-labels
        combined_y[len(y_train_encoded):] = pseudo_labels

        # Fit the model with updated pseudo-labels
        self_training_model.fit(combined_X, combined_y)

        # Evaluate the classifier after self-training
        y_pred = self_training_model.predict(X_val_processed)
        acc = accuracy_score(y_val_encoded, y_pred)

        # Convert y_pred back to original labels for reporting
        y_pred_original = le.inverse_transform(y_pred)
        y_val_original = le.inverse_transform(y_val_encoded)

        print(f"Results for {name}:")
        print("Accuracy:", acc)
        print("Classification Report:")
        print(classification_report(y_val_original, y_pred_original))
        print("Confusion Matrix:")
        print(confusion_matrix(y_val_original, y_pred_original))
        print("Pseudo-labeled Count:", pseudo_labeled_count)  # Print the number of pseudo-labeled samples
        print("-" * 50)

        results.append((name, acc))

    else:
        print(f"Skipping {name} as it does not support predict_proba")

# Sort and display classifiers by accuracy
results.sort(key=lambda x: x[1], reverse=True)
print("Classifier rankings:")
for i, (name, acc) in enumerate(results):
    print(f"{i + 1}. {name}: {acc:.4f}")

# Print the number of pseudo-labeled samples for each classifier
print("Pseudo-labeled Counts:")
for name, count in pseudo_labeled_counts:
    print(f"{name}: {count}")

Results for Random Forest:
Accuracy: 0.9758507135016465
Classification Report:
              precision    recall  f1-score   support

           A       0.99      0.99      0.99       501
           B       0.98      0.94      0.96       703
           C       0.97      1.00      0.98      2388
           D       1.00      0.48      0.65        21
           E       1.00      0.26      0.41        31

    accuracy                           0.98      3644
   macro avg       0.99      0.73      0.80      3644
weighted avg       0.98      0.98      0.97      3644

Confusion Matrix:
[[ 497    1    3    0    0]
 [   1  660   42    0    0]
 [   1    6 2381    0    0]
 [   0    1   10   10    0]
 [   3    3   17    0    8]]
Pseudo-labeled Count: 14973
--------------------------------------------------
Results for XGBoost:
Accuracy: 0.9777716794731065
Classification Report:
              precision    recall  f1-score   support

           A       0.99      0.99      0.99       501
           B

/Users/annabzinkowska/anaconda3/envs/thesis_/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/annabzinkowska/anaconda3/envs/thesis_/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/annabzinkowska/anaconda3/envs/thesis_/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera